In [ ]:
!pip install transformers
!pip install datasets
!pip install --upgrade --no-cache-dir gdown==4.5.4

!gdown 18oZZ4jqRK-uF-Nz6ftRdgNjKix88hrnO
!unzip data_and_models.zip && rm data_and_models.zip

import numpy as np
np.random.seed(11)
import torch
torch.manual_seed(11)
import random
random.seed(11)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Archive:  data_and_models.zip
   creating: data_and_models/
  inflating: __MACOSX/._data_and_models  
  inflating: data_and_models/logistic_model_8.pkl  
  inflating: __MACOSX/data_and_models/._logistic_model_8.pkl  
  inflating: data_and_models/tfidf_44.pkl  
  inflating: __MACOSX/data_and_models/._tfidf_44.pkl  
  inflating: data_and_models/tfidf_8.pkl  
  inflating: __MACOSX/data_and_models/._tfidf_8.pkl  
  inflating: data_and_models/target_corpus.csv  
  inflating: __MACOSX/data_and_models/._target_corpus.csv  
  inflating: data_and_models/logistic_model_44.pkl  
  inflating: __MACOSX/data_an

In [ ]:
from collections import defaultdict
import csv
import pickle
import time

from datasets import load_metric
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaForSequenceClassification, RobertaTokenizerFast, Trainer, TrainingArguments

def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def multi_class_top_one_accuracy(predictions, labels, class_i):
  """
  For each class, calculate the top 1 accuracy. 
  """
  assert len(predictions) == len(labels)
  total = 0
  correct = 0
  for i in range(len(predictions)):
    if labels[i] != class_i:
      continue
    total += 1
    prediction = []
    for j, k in enumerate(predictions[i]):
      prediction.append([j, k]) # k is the value
    prediction.sort(key = lambda x: -x[1])
    if prediction[0][0] == labels[i]:
      correct += 1
  ans = str(round(correct/total, 3))
  if len(ans) < 5:
    ans += "0" * (5-len(ans))
  return ans

class PSCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

start = time.time()
mlength = 512
directory = "./data_and_models/"

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=20,             # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=0,                  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    learning_rate = 2e-5,
    save_strategy= "epoch",
    evaluation_strategy="epoch",
    load_best_model_at_end= True,
    metric_for_best_model="accuracy",
    seed = 11,
)

tasks = {
    "44": {
        "number_of_labels": 42,
         "label_column": 1,
    },
    "8": {
        "number_of_labels": 8,
        "label_column": 2,
    }
}

def compute_task(task):
  index = -1
  classes = {}
  texts = []
  labels = []
  lm_reverse_mapper = {}


  with open(directory + "target_corpus.csv") as doc:
    reader = csv.reader(doc)
    next(reader)
    for row in reader:
      topic = row[tasks[task]["label_column"]]
      if topic not in classes:
        index += 1
        classes[topic] = index
        lm_reverse_mapper[index] = topic.capitalize()
      labels.append(classes[topic])
      texts.append(row[0])

  print("# classes", len(classes))
  X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=625, random_state=11)
  X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=625, random_state=11)
  print(len(X_train), len(X_dev), len(X_test))
  print("# classes in train", len(set(y_train)))
  print("# classes in dev", len(set(y_dev)))
  print("# classes in test", len(set(y_test)))

  tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
  train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=mlength)
  dev_encodings = tokenizer(X_dev, truncation=True, padding=True, max_length = mlength)
  test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length= mlength)

  train_dataset = PSCDataset(train_encodings, y_train)
  dev_dataset = PSCDataset(dev_encodings, y_dev)
  test_dataset = PSCDataset(test_encodings, y_test)

  def model_init():
    return RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=tasks[task]["number_of_labels"])

  trainer = Trainer(
      model_init=model_init,               # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=dev_dataset,            # evaluation dataset
      compute_metrics=compute_metrics,     # compute_metrics
      )
  trainer.train()

  predictions = trainer.predict(test_dataset)


  with open(directory + "logistic_model_" + task + ".pkl", "rb") as doc:
    model = pickle.load(doc)
  with open(directory + "tfidf_" + task + ".pkl", "rb") as doc:
    tokenizer = pickle.load(doc)

  class_mapper = {}
  class_reverse_mapper = {}
  for i, topic in enumerate(model.classes_):
    class_mapper[topic.replace(" ", ".").replace("-", ".")] = i
    class_reverse_mapper[i] = topic

  df = pd.read_csv(directory + "target_corpus.csv")
  df = df[df["text"].isin(X_test)]
  X = df['text']
  Y = list(df["topic_"+task].transform(lambda x: class_mapper[x]))

  Xtfidf = tokenizer.transform(X)

  preds = model.predict(Xtfidf)
  preds = [class_mapper[topic.replace(" ", ".").replace("-", ".")] for topic in preds]
  policy_probs = model.predict_proba(Xtfidf)

  from collections import Counter
  counter = Counter(Y)
  results = []
  for class_i, count in counter.items():
    result = [class_reverse_mapper[class_i].capitalize(), count]
    result.append(multi_class_top_one_accuracy(policy_probs, Y, class_i))
    #sample result:
    #['Political authority', 140, '0.550']
    #['Welfare state expansion', 49, '0.694']
    results.append(result)

  results.sort(key = lambda result: [-result[1], result[0]])

  for index, topic in lm_reverse_mapper.items():
    topic = topic.replace(".", " ")
    if "demographic" in topic:
      topic = "Non-economic demographic groups"
    lm_reverse_mapper[index]=topic

  lm_per_class_predictions = defaultdict(str)
  counter = Counter(y_test)
  for class_i, count in counter.items():
    lm_per_class_predictions[lm_reverse_mapper[class_i]] = multi_class_top_one_accuracy(predictions.predictions, y_test, class_i)
  outputs = []
  for result in results:
    result += [lm_per_class_predictions[result[0]]]
    if float(result[-1]) > float(result[-2]):
      result[-1] = "\\textbf{" + result[-1] + "}" 
    elif float(result[-1]) < float(result[-2]):
      result[-2] = "\\textbf{" + result[-2] + "}" 
    str_result = [str(i) for i in result]
    outputs.append("& " +" & ".join(str_result)+"\\\\")
  return outputs

results = {}
for task in tasks:
  results[task] = compute_task(task)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


# classes 42
2915 625 625
# classes in train 42
# classes in dev 36
# classes in test 35


loading file vocab.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_toke

Epoch,Training Loss,Validation Loss,Accuracy
1,2.430300,2.288778,0.417600
2,1.917100,1.934722,0.516800
3,1.582500,1.823273,0.542400
4,1.362400,1.823743,0.548800
5,0.972300,1.889241,0.523200
6,0.848000,1.888951,0.534400
7,0.573900,2.010903,0.532800
8,0.409200,1.989290,0.553600
9,0.317400,2.029555,0.563200
10,0.230900,2.128701,0.532800


***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-183
Configuration saved in ./results/checkpoint-183/config.json
Model weights saved in ./results/checkpoint-183/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-366
Configuration saved in ./results/checkpoint-366/config.json
Model weights saved in ./results/checkpoint-366/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-549
Configuration saved in ./results/checkpoint-549/config.json
Model weights saved in ./results/checkpoint-549/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-732
Configuration saved in ./results/checkpoint-732/config.json
Model weights saved in ./results/checkpoint-732/pytorch_model.bin
***** Ru

# classes 8
2915 625 625
# classes in train 8
# classes in dev 8
# classes in test 8


loading file vocab.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_toke

Epoch,Training Loss,Validation Loss,Accuracy
1,1.319900,1.205588,0.577600
2,0.978900,1.116255,0.619200
3,0.816300,1.130859,0.633600
4,0.821600,1.280719,0.632000
5,0.496800,1.309494,0.628800
6,0.146600,1.554289,0.609600
7,0.165500,1.672504,0.625600
8,0.178600,1.918652,0.628800
9,0.038300,2.086765,0.638400
10,0.116000,2.294300,0.625600


***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-183
Configuration saved in ./results/checkpoint-183/config.json
Model weights saved in ./results/checkpoint-183/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-366
Configuration saved in ./results/checkpoint-366/config.json
Model weights saved in ./results/checkpoint-366/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-549
Configuration saved in ./results/checkpoint-549/config.json
Model weights saved in ./results/checkpoint-549/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-732
Configuration saved in ./results/checkpoint-732/config.json
Model weights saved in ./results/checkpoint-732/pytorch_model.bin
***** Ru

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at yo

In [ ]:
for task in tasks:
  print("==== " + task + "-Topic Classification ====")
  for result in results[task]:
    print(result)

==== 44-Topic Classification ====
& Political authority & 140 & 0.550 & \textbf{0.657}\\
& Welfare state expansion & 49 & 0.694 & \textbf{0.714}\\
& Democracy & 44 & 0.318 & \textbf{0.341}\\
& No topic & 32 & 0.000 & \textbf{0.438}\\
& Labour groups & 31 & 0.387 & \textbf{0.484}\\
& Education & 26 & \textbf{0.885} & 0.846\\
& Constitutionalism & 24 & 0.000 & \textbf{0.458}\\
& Economic orthodoxy & 21 & 0.238 & \textbf{0.571}\\
& Governmental and administrative efficiency & 21 & 0.238 & 0.238\\
& Technology and infrastructure & 21 & 0.333 & \textbf{0.524}\\
& Law and order & 20 & 0.650 & \textbf{0.700}\\
& Multiculturalism & 19 & 0.632 & \textbf{0.842}\\
& Equality & 18 & \textbf{0.389} & 0.278\\
& Free market economy & 15 & 0.000 & \textbf{0.267}\\
& Economic growth & 13 & 0.615 & \textbf{0.769}\\
& Freedom and human rights & 13 & 0.000 & \textbf{0.231}\\
& Market regulation & 12 & 0.167 & \textbf{0.333}\\
& Traditional morality & 12 & 0.250 & \textbf{0.333}\\
& Military & 11 & 0.727 &

In [ ]:
end = time.time()
print(f"The program took {(end - start) // 60} minutes in total.")

The program took 52.0 minutes in total.


In [ ]:
from google.colab import runtime
runtime.unassign()